In [1]:
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException

import asyncio
from playwright.async_api import async_playwright
from playwright.sync_api import sync_playwright

import time
import nest_asyncio


In [30]:
# Set the directory path
directory = "archives"

# Threshold filename
threshold_filename = "2025-04-15.md"

# Collect all lines from valid files
data = []

# Iterate over files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".md") and filename > threshold_filename:
        filepath = os.path.join(directory, filename)
        with open(filepath, "r", encoding="utf-8") as f:
            lines = f.readlines()[7:]  # skip first 6 lines (index 0-5)
            for line in lines:
                data.append({"filename": filename, "line": line.strip()})
                

# Convert to DataFrame
df = pd.DataFrame(data)

In [22]:
df = df.sort_values(by="filename", ascending=True).reset_index(drop=True)

In [23]:
df.head()

,filename,line
0,2025-04-16.md,1. [第一次见这么接地气的富二代](https://s.weibo.com//weibo?...
1,2025-04-16.md,1. [特朗普上一任期被指让美减少7.5万岗位](https://s.weibo.com//...
2,2025-04-16.md,1. [独行侠不后悔交易东契奇](https://s.weibo.com//weibo?q=...
3,2025-04-16.md,1. [苹果两款设备被列入过时产品](https://s.weibo.com//weibo?...
4,2025-04-16.md,1. [檀健次hi6录制人气](https://s.weibo.com//weibo?q=%...


In [24]:
df.to_csv("resou.csv", index=False)

In [79]:
df['text'] = df.line.apply(lambda x: re.findall(r"\[(.*?)\]", x)[0] if re.findall(r"\[(.*?)\]", x) else None)
df.head()
df.to_csv("resou.csv", index=False)

In [ ]:
# selunium
def get_ref_detail(url, topic, date):
    # each page
    page_number = 1
    driver.get(url)
    time.sleep(1)
    r_d = []
    while True: 
        print(f"Scraping page {page_number}...")

        # Click all "expand" buttons to reveal full text
        expand_buttons = driver.find_elements(By.CSS_SELECTOR, "a[action-type='fl_unfold']")
        for btn in expand_buttons:
            try:
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(0.5)
            except Exception:
                pass

        # soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = driver.find_elements(By.CLASS_NAME, "card-wrap")
        # cards = soup.select("div.card-wrap")
        
        if len(cards) == 0:
            print("No cards found on this page.")
            r_d.append({
                "account_name": "",
                "account_link": "",
                "post_date": "",
                "forward": "",
                "comment": "",
                "like": "",
                "text": "",
                "image": "",
                "video": "",
                "vip": "",
                "topic": topic,
                "date": date
            })
            return r_d

        for card in cards:
            try:
                # 账户名
                name_tag = card.select_one("a.name")
                account_name = name_tag.text.strip() if name_tag else ""
                # 账户链接
                account_link = "https:" + name_tag["href"] if name_tag else ""

                # 发布时间
                post_date_tag = card.select_one(".from a[target]")
                post_date = post_date_tag.text.strip() if post_date_tag else ""
                # 转发数
                forward_tag = card.select_one("a[action-type='feed_list_forward']")
                forward = forward_tag.text.strip() if forward_tag else "0"
                # 评论数
                comment_tag = card.select_one("a[action-data*='value=weibo_h_1_p_p']")
                comment = comment_tag.text.strip() if comment_tag else "0"
                # 点赞数
                like_tag = card.select_one("span.woo-like-count")
                like = like_tag.text.strip() if like_tag else "0"
                # 正文
                text_tag = card.select_one("p[node-type='feed_list_content']")
                text = text_tag.text.strip() if text_tag else ""
                # 图片：True/False
                image = bool(card.select_one(".media img"))
                # 视频：True/False
                video = bool(card.select_one("div.wbpv-poster"))
                # 是否VIP：True/False
                vip = bool(card.select_one(".user_vip_icon_container img"))

            except Exception as e:
                print("Error parsing card:", e)
                continue
            
            r_d.append({
                "account_name": account_name,
                "account_link": account_link,
                "post_date": post_date,
                "forward": forward,
                "comment": comment,
                "like": like,
                "text": text,
                "image": image,
                "video": video,
                "vip": vip,
                "topic": topic,
                "date": date
            })
        
        # Try to click "下一页"
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
            driver.execute_script("arguments[0].click();", next_button)
            page_number += 1
        except Exception:
            print("No more pages or '下一页' button not found.")
            return r_d

# driver.quit()


In [76]:
# playwright
def get_ref_detail(url, topic, date):
    # each page
    page_number = 1
    # driver.get(url)
    # time.sleep(1)
    r_d = []
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=False, executable_path="/Applications/Google Chrome.app/Contents/MacOS/Google Chrome",args=["--mute-audio"])
    # async with async_playwright() as p:
    #     browser = await p.chromium.launch(headless=True)
    #     context = await browser.new_context()
    #     page = await context.new_page()
    #     await page.goto(url)
    #     await page.wait_for_timeout(3000)
        context = browser.new_context()
        page = context.new_page()
        page.goto(url)
    
    while True: 
        print(f"Scraping page {page_number}...")
        # wait for page to load
        page.wait_for_selector('.card-wrap')

        # Expand all collapsed posts
        expand_buttons = page.query_selector_all("a[action-type='fl_unfold']")
        for btn in expand_buttons:
            try:
                btn.click()
                page.wait_for_timeout(300)
            except:
                continue

        # Find all post cards
        cards = page.query_selector_all("div.card-wrap")
        
        if len(cards) == 0:
            print("No cards found on this page.")
            r_d.append({
                "account_name": "",
                "account_link": "",
                "post_date": "",
                "forward": "",
                "comment": "",
                "like": "",
                "text": "",
                "image": "",
                "video": "",
                "vip": "",
                "topic": topic,
                "date": date
            })
            return r_d
        
        for card in cards:
            try:
                name_elem = card.query_selector("a.name")
                account_name = name_elem.inner_text() if name_elem else ""
                account_link = name_elem.get_attribute("href") if name_elem else ""

                date_elem = card.query_selector(".from a[target]")
                post_date = date_elem.inner_text() if date_elem else ""

                forward_elem = card.query_selector("a[action-type='feed_list_forward']")
                forward = forward_elem.inner_text() if forward_elem else "0"

                comment_elem = card.query_selector("a[action-data*='value=weibo_h_1_p_p']")
                comment = comment_elem.inner_text() if comment_elem else "0"

                like_elem = card.query_selector("span.woo-like-count")
                like = like_elem.inner_text() if like_elem else "0"

                text_elem = card.query_selector("p[node-type='feed_list_content']")
                text = text_elem.inner_text() if text_elem else ""

                has_image = card.query_selector(".media img") is not None
                has_video = card.query_selector("div.wbpv-poster") is not None
                is_vip = card.query_selector(".user_vip_icon_container img") is not None
            
                r_d.append({
                    "account_name": account_name,
                    "account_link": account_link,
                    "post_date": post_date,
                    "forward": forward,
                    "comment": comment,
                    "like": like,
                    "text": text,
                    "image": has_image,
                    "video": has_video,
                    "vip": is_vip,
                    "topic": topic,
                    "date": date
                })

            except Exception as e:
                print(f"Error on a card: {e}")
                continue
    
        # Try to click "下一页"
        # Check for and click "下一页"
        next_btn = page.query_selector("a.next")
        if next_btn:
            next_btn.click()
            current_page += 1
        else:
            print("No more pages.")
            return r_d
            break

    browser.close()

In [ ]:
BASE_URL = "https://s.weibo.com/aisearchmore?q={topic}&res_type=ref_blog&page={page}"

options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

ref_data = []
for index, row in df.iloc[1:].iterrows():
    topic = row['text']
    url = BASE_URL.format(topic=topic, page=1)
    print(f"Fetching data for topic: {topic}")
    
    # Set up headless Chrome
    # options = Options()
    # options.add_argument("--headless")
    # driver = webdriver.Chrome(options=options)
    try:
        new = get_ref_detail(url, topic, row['filename'][:-3])
        ref_data.extend(new)
    except Exception as e:
        print(f"Error fetching data for topic {topic}: {e}")
        break
    # Make a request to the URL
    # response = requests.get(url)
    # soup = BeautifulSoup(response.text, 'html.parser')
    # page_range = len(soup.find('ul', class_='s-scroll').find_all('li')) if soup.find('ul', class_='s-scroll') else 0
    # print(url)
    # print(response.status_code)
    # print(response.text)
    # if page_range == 0:
    #     ref_data.append({
    #             "account_name": None,
    #             "account_link": None,
    #             "post_date": None,
    #             "forward": None,
    #             "comment": None,
    #             "like": None,
    #             "text": None,
    #             "image": None,
    #             "video": None,
    #             "vip": None,
    #             "topic": topic,
    #             "date": row['filename'][:-3]
    #         })
    # # pages
    # for i in range(page_range):
    #     # response = requests.get(url, params={'page': i})
    #     # if response.status_code == 200:

    #     # scarpe each page
    #     ref_detail = get_ref_detail(BASE_URL.format(topic=topic, page=i))
    #     ref_detail['topic'] = topic
    #     ref_detail['date'] = row['filename'][:-3]

    #     ref_data.append(ref_detail)

    # else:
    #     print(f"Failed to fetch data for topic: {topic}")

    break

ref_df = pd.DataFrame(ref_data)
# driver.quit()




Fetching data for topic: 太原爆炸已致1死21伤2失联
Error fetching data for topic 太原爆炸已致1死21伤2失联: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.


In [53]:
ref_df

,account_name,account_link,post_date,forward,comment,like,text,image,video,vip,topic,date
0,,,,,,,,,,,太原爆炸已致1死21伤2失联,2025-04-30


In [83]:
# Read the file
with open("account2.json", "r") as f:
    content = f.read()

# Replace single quotes with double quotes
corrected = content.replace("'", '"')

# Save the corrected version
with open("account2_fixed.json", "w") as f:
    f.write(corrected)

scraping

In [6]:

"欠200万老赖冒用身份坐高铁被拘5日" in true['topic'].values

True

In [2]:
true = pd.read_csv("modeling_data_true.csv")
true.groupby('topic').count().reset_index()[['topic', 'account_name']]

,topic,account_name
0,12306回应动车停摆乘客被困4小时,47
1,80后国企财务成短剧恶婆婆专业户,43
2,AG七连进决赛,94
3,EDG回家,100
4,Jennie人间香奈儿,170
...,...,...
95,预计2025年黄金均价将达3065美元,107
96,饿了么害我一天三杯奶茶,106
97,马斯克称地球会被太阳焚化,81
98,马斯克被爆踢出局,145


In [2]:
ref = pd.read_csv("ref_data_April_30.csv")
# num_unique_combinations = ref[['topic', 'topic-date']].drop_duplicates().shape[0]
ref.head()

,account_name,account_link,post_date,forward,comment,like,text,image,video,vip,topic,topic-date
0,人民日报,//weibo.com/2803301701?refer_flag=1001030103_,04月30日 20:42,4577,0,8515,\n 【我的追梦关键词｜#陈梦说要全力以赴不留遗憾#】...,False,True,True,陈梦说要全力以赴不留遗憾,2025-04-30
1,罗琦真棒,//weibo.com/5436029657?refer_flag=1001030103_,04月30日 20:27,5,0,895,#陈梦说要全力以赴不留遗憾#我不是看不上陈梦，我觉得她现在有点更扁向于娱乐化了，说实话也没看...,False,False,True,陈梦说要全力以赴不留遗憾,2025-04-30
2,冷墨的时间,//weibo.com/5480003100?refer_flag=1001030103_,04月30日 19:57,转发,0,33,\n 《乒乓世界》对陈梦的采访，被她那句“全力以赴不留...,False,True,True,陈梦说要全力以赴不留遗憾,2025-04-30
3,阚心童,//weibo.com/5664150156?refer_flag=1001030103_,04月30日 20:28,13,0,573,#陈梦说要全力以赴不留遗憾#\n\n没人脉 没大团队就是这样的 只有空降代言和访谈 ​,True,False,True,陈梦说要全力以赴不留遗憾,2025-04-30
4,小羊也_,//weibo.com/7743001624?refer_flag=1001030103_,04月30日 21:30,转发,0,74,#陈梦说要全力以赴不留遗憾# 不管是工作 学习 梦想 都要全力以赴地去拼 去冲 不给自己留下...,True,False,True,陈梦说要全力以赴不留遗憾,2025-04-30


In [18]:
scraped_resou = pd.read_csv("scraped_resou.csv")
unique_topics = ref[~ref['topic'].isin(scraped_resou['text'].to_list())]['topic'].drop_duplicates().sample(n=30, random_state=42)
unique_topics = unique_topics.to_list()

In [ ]:
resou = pd.read_csv("resou.csv")
a_30 = resou[(resou['filename'] == "2025-04-30.md") & (resou['text'].isin(unique_topics))]
a_30 = a_30.drop_duplicates(keep="first").reset_index(drop=True)

# entertainment, social, politics, foreign
subject = ['entertainment', 'social', 'social', 'entertainment', 'entertainment', 'entertainment', 'social', 'foreign', 'social', 'social',
            'politics', 'social', 'entertainment', 'entertainment', 'social', 'entertainment', 'politics', 'entertainment', 'entertainment','entertainment',
            'entertainment','entertainment','social','entertainment','social','social','entertainment','entertainment','social','politics']
a_30['subject'] = subject

scraped_resou = pd.concat([scraped_resou, a_30], ignore_index=True)
# scraped_resou.to_csv("scraped_resou_final.csv", index=False)

In [5]:
len(ref)

2505532

In [29]:
len(a_30)

30

In [36]:
a_30_ref_2 = a_30_ref
a_30_ref_2.drop_duplicates(inplace=True)
len(a_30_ref_2)

/var/folders/cn/fww9r0gd1mg20td4q8v12_lsknvlj7/T/ipykernel_5109/1290273516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_30_ref_2.drop_duplicates(inplace=True)


2551

In [37]:
a_30_ref_2.groupby('topic').size().reset_index(name='counts').sort_values(by='counts', ascending=False)

,topic,counts
12,支持内娱女明星都去拍撕拉片,221
27,解放军仪仗队亮相越南阅兵,187
21,神十九返回舱着陆,152
6,国羽拳击手李诗沣,147
28,迪丽热巴从漫画里走出来了,137
25,肖某事件对医生群体造成伤害,127
9,宋亚轩捏张真源鼻子,123
4,协和4加4事件需尽快给公众回应,122
10,屠呦呦当选美国科学院外籍院士,121
11,拒绝肖飞打招呼医生3天涨粉5.3万,104


In [39]:
modeling = pd.read_csv("modeling_data.csv")
modeling = pd.concat([a_30_ref_2, modeling], ignore_index=True)
len(modeling)

172505

In [41]:
modeling.drop_duplicates(inplace=True)

In [63]:
scraped_resou = pd.read_csv("scraped_resou_final.csv")
modeling_2 = pd.merge(scraped_resou[['text', 'subject']], modeling, left_on="text", right_on="topic", how='right').drop(columns=['text_x','ref']).rename(columns={'text_y': 'text'})
modeling_2.groupby('subject').size().reset_index(name='counts').sort_values(by='counts', ascending=False)

,subject,counts
0,entertainment,3742
3,social,2147
1,foreign,983
2,politics,811


In [64]:
modeling_2['referred'] = True
modeling_2.head()

,subject,account_name,account_link,post_date,forward,comment,like,text,image,video,vip,topic,topic-date,referred
0,social,姜汝祥-,//weibo.com/1317335037?refer_flag=1001030103_,05月01日 22:02,462,0,5081,屠呦呦94岁高龄，于2025年4月30日当选为美国国家科学院外籍院士，她在85岁的时候获得诺...,False,True,True,屠呦呦当选美国科学院外籍院士,2025-04-30,True
1,social,凤凰网,//weibo.com/2615417307?refer_flag=1001030103_,04月30日 11:24,381,0,1181,#屠呦呦当选美国科学院外籍院士# 【95岁屠呦呦，当选美国科学院外籍院士】4月30日，美国国...,True,False,True,屠呦呦当选美国科学院外籍院士,2025-04-30,True
2,social,碗吹,//weibo.com/6082100482?refer_flag=1001030103_,04月30日 14:59,1,0,12,#屠呦呦当选美国科学院外籍院士#屠呦呦于2025年4月30日当选美国国家科学院外籍院士，这是...,False,True,True,屠呦呦当选美国科学院外籍院士,2025-04-30,True
3,social,新闻晨报,//weibo.com/1314608344?refer_flag=1001030103_,04月30日 13:00,2219,0,42777,\n 【95岁#屠呦呦当选美国科学院外籍院士#】4月3...,False,True,True,屠呦呦当选美国科学院外籍院士,2025-04-30,True
4,social,知识分子,//weibo.com/5705191799?refer_flag=1001030103_,04月30日 16:42,21,0,69,#屠呦呦当选美国科学院外籍院士# 【屠呦呦当选美国科学院院士丨附屠呦呦专访】当地时间4月30...,True,False,False,屠呦呦当选美国科学院外籍院士,2025-04-30,True


In [65]:
modeling_2.to_csv("modeling_data_true.csv", index=False)

In [66]:
modeling_2.groupby('topic').size().reset_index(name='counts').sort_values(by='counts', ascending=False)

,topic,counts
2,Bin blg回来了,270
45,支持内娱女明星都去拍撕拉片,221
7,丁禹兮孟子义淮水竹亭剧情线来了,189
82,解放军仪仗队亮相越南阅兵,187
67,王一博周深邓为等亮相央视五四晚会,174
...,...,...
86,这就是有温度的中国排面,31
48,日本女孩屡报警遭漠视4个月变白骨,31
16,以谈为幌子搞胁迫讹诈对中国行不通,23
51,景元寰宇因你而齐响,15


In [69]:
# modeling_2[~modeling_2['topic'].isin(scraped_resou['text'].unique().tolist())]
scraped_resou[~scraped_resou['text'].isin(modeling_2['topic'].unique().tolist())]

,filename,line,text,subject
0,2025-05-01.md,1. [彭昱畅这么帅了](https://s.weibo.com//weibo?q=%23%...,彭昱畅这么帅了,entertainment
1,2025-05-01.md,1. [半个娱乐圈都在大风杀](https://s.weibo.com//weibo?q=%...,半个娱乐圈都在大风杀,entertainment
2,2025-05-01.md,1. [赵露思音乐节](https://s.weibo.com//weibo?q=%E8%B...,赵露思音乐节,entertainment
3,2025-05-01.md,1. [周深一开嗓唱出踏风破浪](https://s.weibo.com//weibo?q=...,周深一开嗓唱出踏风破浪,entertainment
4,2025-05-01.md,1. [张艺兴的舞台](https://s.weibo.com//weibo?q=%E5%B...,张艺兴的舞台,entertainment
...,...,...,...,...
110,2025-05-03.md,1. [高中生继承5亿遗产结婚当天坠亡](https://s.weibo.com//weib...,高中生继承5亿遗产结婚当天坠亡,social
125,2025-05-04.md,1. [习近平寄语青年朋友](https://s.weibo.com//weibo?q=%2...,习近平寄语青年朋友,politics
131,2025-05-04.md,1. [习近平给戍边支教志愿者回信](https://s.weibo.com//weibo?...,习近平给戍边支教志愿者回信,politics
142,2025-05-04.md,1. [怀爱国之心立报国之志增强国之能](https://s.weibo.com//weib...,怀爱国之心立报国之志增强国之能,politics


In [44]:
# modeling.to_csv("modeling_data.csv", index=False)
len(modeling)

7640

In [32]:
a_30_ref.topic.unique()

array(['屠呦呦当选美国科学院外籍院士', '麻醉科医生谈肖飞手术离场40分钟', '神十九返回舱着陆', '以前的下属一年后成同级别总监',
       '迪丽热巴从漫画里走出来了', '孙颖莎发言4个小脑袋排排看', '羊角竟然是空心的', '景元寰宇因你而齐响',
       '明确实行婚姻登记全国通办', '宋亚轩捏张真源鼻子', '杨超越拽着王安宇耕了一亩地', '章若楠方否认番位操控',
       '穆雷43分', '支持内娱女明星都去拍撕拉片', '董某莹的跨界疑云有谁在紧张', '唐悠悠的玫瑰被邓家佳戴上了',
       '有个付出型朋友是什么体验', '拒绝肖飞打招呼医生3天涨粉5.3万', '人鱼的眼泪作者回应', '曾黎团队傲慢公关丢了人心',
       '肖某事件对医生群体造成伤害', '解放军仪仗队亮相越南阅兵', '林诗栋蒯曼和朝鲜组合同区', '五一北方穿外套南方要防暑',
       '王辰', 'papi酱梦女', '孙颖莎大藤沙月同区', '国羽拳击手李诗沣', '协和4加4事件需尽快给公众回应',
       '新加坡前外交官在澡堂偷拍男学生'], dtype=object)

In [28]:
len(modeling)

169954

In [26]:
a_30_ref.head()

,account_name,account_link,post_date,forward,comment,like,text,image,video,vip,topic,topic-date
546,姜汝祥-,//weibo.com/1317335037?refer_flag=1001030103_,05月01日 22:02,462,0,5081,屠呦呦94岁高龄，于2025年4月30日当选为美国国家科学院外籍院士，她在85岁的时候获得诺...,False,True,True,屠呦呦当选美国科学院外籍院士,2025-04-30
547,凤凰网,//weibo.com/2615417307?refer_flag=1001030103_,04月30日 11:24,381,0,1181,#屠呦呦当选美国科学院外籍院士# 【95岁屠呦呦，当选美国科学院外籍院士】4月30日，美国国...,True,False,True,屠呦呦当选美国科学院外籍院士,2025-04-30
548,碗吹,//weibo.com/6082100482?refer_flag=1001030103_,04月30日 14:59,1,0,12,#屠呦呦当选美国科学院外籍院士#屠呦呦于2025年4月30日当选美国国家科学院外籍院士，这是...,False,True,True,屠呦呦当选美国科学院外籍院士,2025-04-30
549,新闻晨报,//weibo.com/1314608344?refer_flag=1001030103_,04月30日 13:00,2219,0,42777,\n 【95岁#屠呦呦当选美国科学院外籍院士#】4月3...,False,True,True,屠呦呦当选美国科学院外籍院士,2025-04-30
550,知识分子,//weibo.com/5705191799?refer_flag=1001030103_,04月30日 16:42,21,0,69,#屠呦呦当选美国科学院外籍院士# 【屠呦呦当选美国科学院院士丨附屠呦呦专访】当地时间4月30...,True,False,False,屠呦呦当选美国科学院外籍院士,2025-04-30


In [ ]:
modeling = pd.read_csv("modeling_data.csv")


modeling data validation

In [11]:
resou[resou['filename'].str.startswith('2025-05-01')].head(30)

,filename,line,text
3530,2025-05-01.md,1. [阳光明媚护士和癌症晚期爷爷](https://s.weibo.com//weibo?...,阳光明媚护士和癌症晚期爷爷
3536,2025-05-01.md,1. [五一假期发朋友圈的小妙招](https://s.weibo.com//weibo?q...,五一假期发朋友圈的小妙招
3535,2025-05-01.md,1. [白银黄金大跳水](https://s.weibo.com//weibo?q=%23%...,白银黄金大跳水
3531,2025-05-01.md,1. [周迅50岁素颜状态](https://s.weibo.com//weibo?q=%2...,周迅50岁素颜状态
3534,2025-05-01.md,1. [女生自创发型正面刘海后脑勺寸头](https://s.weibo.com//weib...,女生自创发型正面刘海后脑勺寸头
3521,2025-05-01.md,1. [王子奇五分钟斯诺克无剪辑版](https://s.weibo.com//weibo?...,王子奇五分钟斯诺克无剪辑版
3522,2025-05-01.md,1. [七号打歌中心](https://s.weibo.com//weibo?q=%23%E...,七号打歌中心
3523,2025-05-01.md,1. [五一档新片总票房破1亿](https://s.weibo.com//weibo?q=...,五一档新片总票房破1亿
3524,2025-05-01.md,1. [华为PuraX](https://s.weibo.com//weibo?q=%23%...,华为PuraX
3525,2025-05-01.md,1. [马丽感谢沈腾粉丝和沈马cp粉](https://s.weibo.com//weibo...,马丽感谢沈腾粉丝和沈马cp粉


In [10]:
resou = pd.read_csv("resou.csv")
resou.sort_values(by="filename", ascending=True, inplace=True)
resou

,filename,line,text
7792,2025-04-16.md,1. [第一次见这么接地气的富二代](https://s.weibo.com//weibo?...,第一次见这么接地气的富二代
7867,2025-04-16.md,1. [特朗普上一任期被指让美减少7.5万岗位](https://s.weibo.com//...,特朗普上一任期被指让美减少7.5万岗位
7866,2025-04-16.md,1. [独行侠不后悔交易东契奇](https://s.weibo.com//weibo?q=...,独行侠不后悔交易东契奇
7865,2025-04-16.md,1. [苹果两款设备被列入过时产品](https://s.weibo.com//weibo?...,苹果两款设备被列入过时产品
7864,2025-04-16.md,1. [檀健次hi6录制人气](https://s.weibo.com//weibo?q=%...,檀健次hi6录制人气
...,...,...,...
6525,2025-05-07.md,1. [微信支付一直在花我的钱](https://s.weibo.com//weibo?q=...,微信支付一直在花我的钱
6526,2025-05-07.md,1. [王蓉终于找对赛道了](https://s.weibo.com//weibo?q=%E...,王蓉终于找对赛道了
6527,2025-05-07.md,1. [默茨未能当选德国总理](https://s.weibo.com//weibo?q=%...,默茨未能当选德国总理
6507,2025-05-07.md,1. [特朗普承认不排除任期内经济萎缩](https://s.weibo.com//weib...,特朗普承认不排除任期内经济萎缩


In [ ]:
# date distribution
ref.groupby('topic-date').size().reset_index(name='counts')

,topic-date,counts
0,2025-04-30,2505532


In [6]:
ref.columns

Index(['account_name', 'account_link', 'post_date', 'forward', 'comment',
       'like', 'text', 'image', 'video', 'vip', 'topic', 'topic-date'],
      dtype='object')

In [4]:
num_unique_combinations

323

In [6]:
len(ref['topic'].unique())

111

In [3]:
true1 = pd.read_csv("modeling_data_true.csv")
true2 = pd.read_csv("modeling_data_true_2.csv")
true = pd.concat([true1, true2], ignore_index=True)
true = true.drop_duplicates(keep="first").reset_index(drop=True)
len(true)

17490

In [4]:
true.to_csv("modeling_data_true.csv", index=False)

In [11]:
true = pd.read_csv("modeling_data_true.csv")
false = pd.read_csv("testing_modeling_data.csv")

In [5]:
len(true)

17490

In [7]:
len(false.topic.unique())

208

In [9]:
len(true.topic.unique())

194

In [12]:
diff = false[~false['topic'].isin(true['topic'].unique().tolist())]

diff.topic.unique()

array(['谁能管管商场厕所', '坚决克服麻痹思想狠抓工作落实', '怀爱国之心立报国之志增强国之能', '习近平给戍边支教志愿者回信',
       '习近平寄语青年朋友', '总书记寄望新时代青年', '为中国式现代化建设贡献青春力量', '总书记关怀劳动者的温暖瞬间',
       '商务部回应中美经贸对话磋商情况', '科学谋划好十五五总书记这样强调', '美国人提醒美国不可能取代中国',
       '3个数字前瞻习主席俄罗斯之行'], dtype=object)